In [8]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [9]:
user_base = pd.read_csv('bases/users.csv')
task_base = pd.read_csv('bases/tasks.csv')
aloc_base = pd.read_csv('bases/alocacao.csv')

In [ ]:
task_base.columns

In [11]:
task_base.drop(['starts_at','actual_starts_at'], axis=1, inplace=True)

In [ ]:
aloc_base.columns

In [13]:
import numpy as np
def gerar_valores_normais(media, desvio_padrao, tamanho):
    valores_normais = np.random.normal(loc=media, scale=desvio_padrao, size=tamanho)
    return valores_normais

In [15]:
task_base.fillna('2026-12-31 00:00:00', inplace=True)

***RODAR A PARTIR DAKI***

In [ ]:
task_base['delay'] = 0
for index in task_base.index:
    if type(task_base['actual_ends_at'][index]) == float:
        print("Error linha: ", index)
        break
    ends_at = datetime.strptime(task_base['ends_at'][index][:10], '%Y-%m-%d')
    actual_end = datetime.strptime(task_base['actual_ends_at'][index][:10], '%Y-%m-%d')

    task_base['ends_at'][index] = datetime.strftime(ends_at, '%d-%m-%Y')
    task_base['actual_ends_at'][index] = datetime.strftime(actual_end, '%d-%m-%Y')
    
    delay = (ends_at - actual_end)
    task_base['delay'][index] = delay.days
    

In [ ]:
task_base['priority'] = 0

for index in task_base.index:
    priority = round(gerar_valores_normais(2, 1, 1)[0]) % 5
    task_base['priority'][index] = priority

In [ ]:
task_base['priority'].value_counts()

In [19]:
task_base.rename(columns={'id':'task_id'}, inplace=True)

In [20]:
task_base.shape

(2000, 6)

In [21]:
task_base = pd.merge(task_base, aloc_base, on='task_id', how='left')

In [ ]:
task_base

**Converte o uuid para UserX**

In [23]:
userSet = set(task_base['user_id'])
count = 0
for user in userSet:
    task_base['user_id'].replace(user, f'user{count}', inplace=True)
    count += 1

In [ ]:
task_base

***Cria a performance***

In [ ]:
task_base['performance'] = 0
for index in task_base.index:
    perc = task_base['percentage'][index]
    delay = task_base['delay'][index]

    performance = delay * (perc/100)

    task_base['performance'][index] = performance

In [ ]:
task_base['performance'].hist(bins=50)

***Remove colunas que não serão utilizadas***

In [27]:
task_base.columns

Index(['task_id', 'name', 'ends_at', 'actual_ends_at', 'delay', 'priority',
       'user_id', 'percentage', 'performance'],
      dtype='object')

In [28]:
task_base.drop(['task_id', 'name', 'ends_at', 'actual_ends_at', 'delay', 'percentage'], axis=1, inplace=True)

In [ ]:
task_base

***Faz media de performance por user***

In [30]:
new_base = pd.DataFrame(columns=['user_id', 'priority', 'performance'])
substitui_null = -100
for priority in range(5):
    for user in task_base['user_id'].unique():
        perf_list = task_base.loc[(task_base['user_id'] == user) & (task_base['priority'] == priority), 'performance']
        media = perf_list.mean()
        if len(perf_list) == 0:
            sum = 0
            for i in range(5):
                perf_list = task_base.loc[(task_base['user_id'] == user) & (task_base['priority'] == i), 'performance']
                if len(perf_list) == 0:
                    sum += 0
                else:
                    sum += perf_list.mean()
            media = sum/5
            
        
        new_base.loc[len(new_base)] ={'user_id': user, 'priority': priority, 'performance': media}


In [ ]:
new_base['performance'].hist(bins=100)

In [ ]:
new_base

In [33]:
new_base = new_base.sort_values(by=['user_id', 'priority'])

In [34]:
new_base.reset_index(drop=True, inplace=True)

In [ ]:
new_base

In [76]:
n = 450
size = n
if size != -1:
    size *= 5
else:
    size = len(new_base)
new_base[0:size].to_csv(f'bases/final_base{n}.csv', index=False)